In [1]:
import os
import math
import pandas as pd
import numpy as np
from pathlib import Path
import shutil
import argparse
from typing import List, Union
import matplotlib.pyplot as plt
import yaml
from datetime import datetime
import logging

import torch
from torch import Tensor, nn
from torch.types import Device, _size
from torch.nn.parameter import Parameter, UninitializedParameter
from torch.nn import init
from torch.utils.data import Dataset
from torch.utils.data import ConcatDataset
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from collections import OrderedDict

# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import train_test_split
from configs.config import configs

from models.resnet1D import *
from models.pe import PositionalEncoding
from models.transformer_encoder import transformer_classifier


2024-10-29 16:59:01.107034: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-29 16:59:01.336021: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-29 16:59:01.828538: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Evaluate norm of tensor

### Initilization

In [2]:
# Init logging
import logging

logger = logging.getLogger(__name__)  # Use the current module's name
logging.basicConfig(level=logging.DEBUG)
# logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler()
# formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
# handler.setFormatter(formatter)
logger.addHandler(handler)
acc_example = 0.95  # Replace with your actual accuracy calculation
logger.info(f"Current accuracy: %{acc_example}")  # Log as info
# logger.debug("Current accuracy: %.2f", accuracy)  # Log as info

Current accuracy: %0.95
INFO:__main__:Current accuracy: %0.95


In [3]:
parser = argparse.ArgumentParser()
parser.add_argument("config_file", metavar="FILE", help="config file")
# parser.add_argument('--run-dir', metavar='DIR', help='run directory')
# parser.add_argument('--pdb', action='store_true', help='pdb')
args = parser.parse_args(args=['configs/abnormal_12000.yml'])
# args, opts = parser.parse_known_args()
# f = 'configs/eeg_pt.yml'
with open(args.config_file, 'r') as file:
    Configs = yaml.safe_load(file)

### Load dataset

In [4]:
# transform signal
def transform(data:Tensor, mean:Tensor, std:Tensor):
    normalized_data = (data - mean) / std
    return normalized_data

In [5]:
class customDataset(Dataset):
    def __init__(self, data_dir:str, label_dir:str, label_dict:dict, mean: list, std: list, transform=None):
#         self.annotations = pd.read_csv(label_dir)
        self.data_dir = data_dir   # './data/seg_csv/train'
        self.label_dir = label_dir
        self.transform = transform
        self.files = os.listdir(self.data_dir)
        self.annotations = pd.read_csv(self.label_dir)
        self.label_dict = label_dict
        self.mean = torch.tensor(mean)
        self.std = torch.tensor(std)
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, index):
        data_path = os.path.join(self.data_dir, self.files[index])
        data = pd.read_csv(data_path)
        data = torch.tensor(data.values, dtype=torch.float32)
        file_name = self.files[index]
        
#         label = torch.tensor(int(self.label_dict[self.annotations.iloc[index,1]]))
        label = self.annotations.loc[self.annotations['csv_file'] == file_name, ['label']].to_string(index=False,header=False)
        label = torch.tensor(int(self.label_dict[label]))
        
        
        if self.transform:
            data = self.transform(data, self.mean, self.std)
            
        return (data, label, file_name)

### Define model

In [6]:
class model(nn.Module):
    def __init__(self, input_size: int, n_channels: int, model_hyp: dict, classes: int):
        super(model, self).__init__()
        self.ae = resnet18(n_channels=n_channels, groups=n_channels, num_classes=classes, d_model=model_hyp['d_model'])
        self.transformer_encoder = transformer_classifier(input_size, n_channels, model_hyp, classes)
#         self.mlp = nn.Sequential(
#             nn.Linear(n_channels*model_hyp['d_model'], n_channels*model_hyp['d_model']//8),
#             nn.ReLU(),
#             nn.Linear(n_channels*model_hyp['d_model']//8, n_channels*model_hyp['d_model']//32),
#             nn.ReLU(),
#             nn.Linear(n_channels*model_hyp['d_model']//32, classes),
#         )
        self.reset_parameters()
        
    def reset_parameters(self):
        r"""Initiate parameters in the model."""
        
        for p in self.parameters():
            if p.dim() > 1:
#                 logger.debug(p.shape)
                nn.init.xavier_uniform_(p)
                    
        for m in self.modules():
#             print(m)
            if isinstance(m, nn.Conv1d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
        
            elif isinstance(m, nn.LayerNorm):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
        print('Complete initiate parameters')

    def forward(self, x):
#         z = self.pe(x)
        z = x.transpose(-1,-2)
        z = self.ae(z)
        y = transformer_encoder(z)
#         z = torch.flatten(z, 1)
#         y = self.mlp(z)
#         z = self.transformer_encoder(z)
        return y

In [7]:
classifier = model(input_size=Configs['input_size'],
                                        n_channels = Configs['n_channels'],
                                        model_hyp=Configs['model'],
                                        classes=len(Configs['dataset']['classes'])).to('cuda')

/home/yossi/anaconda3/envs/python38/lib/python3.8/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Complete initiate parameters


In [8]:
classifier

model(
  (ae): ResNet(
    (conv1): Conv1d(19, 152, kernel_size=(64,), stride=(2,), padding=(3,), groups=19, bias=False)
    (avgpool1d): AdaptiveAvgPool1d(output_size=6000)
    (ln1): LayerNorm((6000,), eps=1e-05, elementwise_affine=True)
    (relu): ReLU(inplace=True)
    (avgpool_1): AvgPool1d(kernel_size=(3,), stride=(2,), padding=(1,))
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv1d(152, 152, kernel_size=(16,), stride=(1,), padding=(1,), groups=19, bias=False)
        (avgpool_1): AdaptiveAvgPool1d(output_size=3000)
        (ln1): LayerNorm((3000,), eps=1e-05, elementwise_affine=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv1d(152, 152, kernel_size=(16,), stride=(1,), padding=(1,), groups=19, bias=False)
        (avgpool_2): AdaptiveAvgPool1d(output_size=3000)
        (ln2): LayerNorm((3000,), eps=1e-05, elementwise_affine=True)
      )
      (1): BasicBlock(
        (conv1): Conv1d(152, 152, kernel_size=(16,), stride=(1,), padding=(1,), gr

In [9]:
train_data_dir = Configs['dataset']['train_data_dir']
train_label_dir = Configs['dataset']['train_label_dir']

val_data_dir = Configs['dataset']['val_data_dir']
val_label_dir = Configs['dataset']['val_label_dir']

label_dict = Configs['dataset']['classes']

mean =  Configs['dataset']['mean']
std =  Configs['dataset']['std']

train_dataset = customDataset(data_dir=train_data_dir,
                              label_dir=train_label_dir,
                              label_dict=label_dict,
                              mean=mean,
                              std=std,
                             transform=transform)
val_dataset = customDataset(data_dir=val_data_dir,
                            label_dir=val_label_dir,
                            label_dict=label_dict,
                            mean=mean,
                            std=std,
                           transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=4,
                              shuffle=Configs['dataset']['shuffle'], 
                          num_workers=Configs['dataset']['num_workers'], pin_memory=True)

eval_loader = DataLoader(dataset=val_dataset, num_workers=Configs['dataset']['num_workers'], 
                         shuffle=Configs['dataset']['shuffle'], pin_memory=True)

# classifier = model(input_size=Configs['input_size'],
#                                     n_channels = Configs['n_channels'],
#                                     model_hyp=Configs['model'],
#                                     classes=len(Configs['dataset']['classes'])).to('cuda')

# input_layer = nn.Sequential(
# #         nn.Embedding(num_embeddings=10000, embedding_dim=512),
# #         PositionalEncoding(d_model=512, dropout=0.1, max_len=5000)
#     PositionalEncoding(d_model=Configs['n_channels'], max_len=Configs['input_size'])
# ).to('cuda')

classifier = model(input_size=Configs['input_size'],
                                        n_channels = Configs['n_channels'],
                                        model_hyp=Configs['model'],
                                        classes=len(Configs['dataset']['classes'])).to('cuda')
optimizer = torch.optim.Adam(classifier.parameters(),betas=(0.9,0.999),lr=Configs['optimizer']['init_lr'])
criterion = nn.CrossEntropyLoss()
writer = SummaryWriter(Configs['tensorboard']['runs_dir']+'train_board')    # Initilize tensorflow

/home/yossi/anaconda3/envs/python38/lib/python3.8/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Complete initiate parameters


/home/yossi/.local/lib/python3.8/site-packages/transformers/utils/generic.py:482: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [10]:
dir(train_dataset)

['__add__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_is_protocol',
 'annotations',
 'data_dir',
 'files',
 'label_dict',
 'label_dir',
 'mean',
 'std',
 'transform']

In [11]:
iter_train = iter(train_loader)
data, target, file = next(iter_train)
data = data.to('cuda')
target = target.to('cuda')
print(file)

['aaaaaocl_s005_t004_1.csv', 'aaaaalto_s001_t001_2.csv', 'aaaaalip_s001_t000_5.csv', 'aaaaakkg_s001_t000_2.csv']


In [12]:
data[1][1179][:]

tensor([ 0.0063,  0.0053,  0.0067,  0.0070,  0.0035,  0.0060,  0.0055,  0.0077,
         0.0067,  0.0046,  0.0072,  0.0061,  0.0087,  0.0071,  0.0093,  0.0062,
         0.0021,  0.0039, -0.0014], device='cuda:0')

In [13]:
out = classifier(data)
loss = criterion(out, target)
probabilities = torch.softmax(out, dim=1)  # Apply softmax to get probabilities
#             _, predicted = torch.max(probabilities, 1)  # Get the predicted class
print(out)
print(probabilities)
print(target)
print(loss)

DEBUG:models.resnet1D:data shape is: torch.Size([4, 19, 12000])
DEBUG:models.resnet1D:data shape after conv1: torch.Size([4, 152, 5972])
DEBUG:models.resnet1D:data shape after ln1: torch.Size([4, 152, 6000])
DEBUG:models.resnet1D:data shape after avgpool: torch.Size([4, 152, 3000])
DEBUG:models.resnet1D:layer1
DEBUG:models.resnet1D:data shape after sub-conv1-3x3: torch.Size([4, 152, 2987]), groups=19
DEBUG:models.resnet1D:data shape after sub-ln1: torch.Size([4, 152, 3000])
DEBUG:models.resnet1D:data shape after sub-relu1: torch.Size([4, 152, 3000])
DEBUG:models.resnet1D:data shape after sub-conv2-3x3: torch.Size([4, 152, 2987]), groups=19
DEBUG:models.resnet1D:data shape after sub-ln2: torch.Size([4, 152, 3000])
DEBUG:models.resnet1D:data shape after residual relu+res: torch.Size([4, 152, 3000])
DEBUG:models.resnet1D:data shape after sub-conv1-3x3: torch.Size([4, 152, 2987]), groups=19
DEBUG:models.resnet1D:data shape after sub-ln1: torch.Size([4, 152, 3000])
DEBUG:models.resnet1D:dat

NameError: name 'transformer_encoder' is not defined

In [ ]:
classifier.ae

#### Check eval results

### Dataset

In [ ]:
class customDataset(Dataset):
    def __init__(self, data_dir:str, label_dir:str, label_dict:dict, transform=None):
#         self.annotations = pd.read_csv(label_dir)
        self.data_dir = data_dir   # './data/origin_csv/train'
        self.label_dir = label_dir
        self.transform = transform
        self.files = os.listdir(self.data_dir)
        self.annotations = pd.read_csv(self.label_dir)
        self.label_dict = label_dict
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, index):
        data_path = os.path.join(self.data_dir, self.files[index])
        data = pd.read_csv(data_path)
        data = torch.tensor(data.values, dtype=torch.float32)
        file_name = self.files[index]
        
        label = torch.tensor(int(self.label_dict[self.annotations.iloc[index,1]]))
        
        if self.transform:
            data = self.transform(data)
            
        return (data, label, file_name)

In [ ]:

# label_dic = {'normal':0, 'abnormal':1}

    
# transform = transforms.Compose([
#     transforms.MinMaxScaler(feature_range=(0, 1)),
#     transforms.ToTensor(),
# ])
# combined_dataset = ConcatDataset([train_dataset, eval_dataset])

### Define model

In [ ]:
### Learning rate update policy
def poly_lr_scheduler(optimizer, init_lr, iter, lr_decay_iter=1,
                      max_iter=0, power=0.9):
    """Polynomial decay of learning rate
        :param init_lr is base learning rate
        :param iter is a current iteration
        :param lr_decay_iter how frequently decay occurs, default is 1
        :param max_iter is number of maximum iterations
        :param power is a polymomial power
    """
    if max_iter == 0:
        raise Exception("MAX ITERATION CANNOT BE ZERO!")
    if iter % lr_decay_iter or iter > max_iter:
        return optimizer
    lr = init_lr * (1 - iter / max_iter) ** power
    logger.info(f'lr=: {lr}')
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return lr

### Train

In [ ]:
def train(Configs:dict):
    train_data_dir = Configs['dataset']['train_data_dir']
    train_label_dir = Configs['dataset']['train_label_dir']

    val_data_dir = Configs['dataset']['val_data_dir']
    val_label_dir = Configs['dataset']['val_label_dir']

    label_dict = Configs['dataset']['classes']
    train_dataset = customDataset(data_dir=train_data_dir,
                                  label_dir=train_label_dir,
                                  label_dict=label_dict)
    val_dataset = customDataset(data_dir=val_data_dir,
                                label_dir=val_label_dir,
                                label_dict=label_dict)
    
    train_loader = DataLoader(dataset=train_dataset, batch_size=Configs['train']['batch_size'],
                                  shuffle=True, num_workers=16, pin_memory=True)

    eval_loader = DataLoader(dataset=val_dataset, num_workers=16, shuffle=True, pin_memory=True)

    classifier = transformer_classifier(input_size=Configs['input_size'], 
                                        n_channels = Configs['n_channels'],
                                        model_hyp=Configs['model'],
                                        classes=len(Configs['dataset']['classes'])).to('cuda')
    optimizer = torch.optim.Adam(classifier.parameters(),betas=(0.9,0.9),lr=Configs['optimizer']['init_lr'])
    criterion = nn.CrossEntropyLoss()
    writer = SummaryWriter(Configs['tensorboard']['runs_dir']+'train_board')    # Initilize tensorflow
    
    ### Warmup training
    warmup_steps = Configs['train']['warmup_steps']
    warmup_step = 0
    min_loss = 1
    while warmup_step < warmup_steps:
        for batch_index, (data,target,_) in enumerate(train_loader, 0):
            if warmup_step < warmup_steps:
        #     for batch_index, data in enumerate(train_loader, 0):
                data, target = data.to('cuda'), target.to('cuda')
                y = classifier(data)
        #         logger.debug(f"y size:{y.shape}, tatget size{target.shape}")
                warmup_loss = criterion(y, target)
                optimizer.zero_grad()
                warmup_loss.backward()
                optimizer.step()
        #         logger.info(f'Epoch: {epoch+1}, Train Loss: {train_loss}')
                logger.info(f"Warmup Step: {warmup_step}, Warmup Loss: {warmup_loss}")
                writer.add_scalar('Warmup Loss', warmup_loss, global_step=warmup_step)
                warmup_step += 1
        
#         if warmup_step%5==0:
#             torch.save(classifier.state_dict(), 
#                        Configs['checkpoint']['checkpoint_dir']+'inte_transformer_params_last.pth')
        if warmup_loss < min_loss:
            torch.save(classifier.state_dict(),
                       Configs['checkpoint']['checkpoint_dir']+'inte_transformer_params_best.pth')
            min_loss = warmup_loss
        
    
    ### train
#     step = 0
#     epochs = Configs['train']['n_epochs']
#     for epoch in range(epochs):
#         # Training loop
#         poly_lr_scheduler(optimizer, init_lr=Configs['optimizer']['init_lr'], iter=epoch, max_iter=epochs)
#         for batch_index, (data,target,_) in enumerate(train_loader, 0):
#             data, target = data.to('cuda'), target.to('cuda')
#             y = classifier(data)
#     #         logger.debug(f"y size:{y.shape}, tatget size{target.shape}")
#             train_loss = criterion(y, target)
#             optimizer.zero_grad()
#             train_loss.backward()
#             optimizer.step()
#     #         logger.info(f'Epoch: {epoch+1}, Train Loss: {train_loss}')
#             logger.info(f"Step: {step}, training Loss: {train_loss}")
#             writer.add_scalar('Training Loss', train_loss, global_step=step)
#             step += 1
        
#         if epoch%5==0:
#             val_loss = 0
#             correct = 0
#             total = 0
#             accuracy = 0
#             with torch.no_grad():
#                 for batch_index, (data,target,_) in enumerate(eval_loader, 0):
#                     data, target = data.to('cuda'), target.to('cuda')
#                     outputs = classifier(data)
#                     loss = criterion(outputs, target)
#                     val_loss += loss.item()
#                     _, predicted = torch.max(outputs, 1)
#                     total += target.size(0)  # Total number of samples
#                     correct += (predicted == target).sum().item()  # Count correct predictions

#             val_loss /= len(eval_loader)
#             accuracy = 100 * correct / total
#             writer.add_scalar('Validation Loss', val_loss, global_step=step)
#             writer.add_scalar('Validation Accuracy', accuracy, global_step=step)
#             logger.info(f'Epoch: {epoch}, Train Loss: {train_loss}, Val Loss: {val_loss:.4f}, Accuracy: {accuracy:.2f}%')

#         torch.save(classifier.state_dict(), 
#                    Configs['checkpoint']['checkpoint_dir']+'inte_transformer_params_latest.pth')
#         if train_loss < min_loss:
#             torch.save(classifier.state_dict(),
#                        Configs['checkpoint']['checkpoint_dir']+'inte_transformer_params_best.pth')
#             min_loss = train_loss

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("config_file", metavar="FILE", help="config file")
# parser.add_argument('--run-dir', metavar='DIR', help='run directory')
# parser.add_argument('--pdb', action='store_true', help='pdb')
args = parser.parse_args(args=['configs/eeg_torch.yml'])
# args, opts = parser.parse_known_args()
# f = 'configs/eeg_pt.yml'
with open(args.config_file, 'r') as file:
    configs = yaml.safe_load(file)
    
# configs['optimizer']['init_lr']
# train(Configs=configs)

In [ ]:
n_channels = 19
ae_1 = nn.Sequential(nn.Conv1d(in_channels=n_channels, out_channels=n_channels, 
                               stride=3, kernel_size=7, dilation=1, groups=n_channels,
                               padding_mode='reflect'),
                     nn.BatchNorm1d(n_channels),
                     nn.ReLU())
ae_2 = nn.Sequential(nn.Conv1d(in_channels=n_channels, out_channels=n_channels, 
                               stride=2, kernel_size=4, dilation=1, groups=n_channels,
                               padding_mode='reflect'),
                     nn.BatchNorm1d(n_channels),
                     nn.ReLU())
ae_3 = nn.Sequential(nn.Conv1d(in_channels=n_channels, out_channels=n_channels, 
                               stride=2, kernel_size=4, dilation=1, groups=n_channels,
                               padding_mode='reflect'),
                     nn.BatchNorm1d(n_channels),
                     nn.ReLU())

# max_pool = nn.MaxPool1d(kernel_size=2, stride=1)
in_data = torch.randn(20, 19, 6000)
out_1 = ae_1(in_data)
out_2 = ae_2(out_1)
out_3 = ae_3(out_2)
# out_data = max_pool(out_data)
out_1.shape, out_2.shape, out_3.shape
# 5660 stride 11

In [ ]:
class transformer_classifier(nn.Module):
    def __init__(self, input_size:int, n_channels:int, model_hyp:dict, classes:int):
        super(transformer_classifier, self).__init__()
        
        self.ae_1 = nn.Sequential(nn.Conv1d(in_channels=n_channels, out_channels=n_channels, 
                                     stride=3, kernel_size=7, dilation=1, groups=n_channels,
                                            padding_mode='reflect'),
                                 nn.BatchNorm1d(n_channels),
                                 nn.ReLU())
        
        self.ae_2 = nn.Sequential(nn.Conv1d(in_channels=n_channels, out_channels=n_channels, 
                                     stride=2, kernel_size=4, dilation=1, groups=n_channels,
                                            padding_mode='reflect'),
                                 nn.BatchNorm1d(n_channels),
                                 nn.ReLU())

        self.ae_3 = nn.Sequential(nn.Conv1d(in_channels=n_channels, out_channels=n_channels, 
                                     stride=2, kernel_size=4, dilation=1, groups=n_channels,
                                            padding_mode='reflect'),
                                 nn.BatchNorm1d(n_channels),
                                 nn.ReLU())
#         self.hidden_size = 597    # need to be calculated every time if you change shape of input
#         self.ae = AutoEncoder(input_size=input_size, hidden_size=model_hyp['d_model'])  
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=model_hyp['d_model'],
                                                        nhead=model_hyp['n_head'])
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=model_hyp['n_layer'])
        self.flatten = nn.Flatten()
        self.linear = nn.Linear(model_hyp['d_model']*n_channels, classes)

    def forward(self, x):
        z = self.ae_1(x)
        z = self.ae_2(z)
        z = self.ae_3(z)
#         z = z[:, :, :1496] 
        logger.debug(f"ae output size: %{z.shape}")
        z = self.transformer_encoder(z)
        logger.debug(f"transformer output size: %{z.shape}")
        z = self.flatten(z)
        logger.debug(f"flatten output size: %{z.shape}")
        y = self.linear(z)
        return y

In [ ]:
in_data = torch.randn(20, 19, 6000).to('cuda')
classifier = transformer_classifier(input_size=6000, n_channels =19,model_hyp=configs['model'],classes=len(configs['dataset']['classes'])).to('cuda')
out = classifier(in_data)

In [ ]:
# target output size of 5
m = nn.AdaptiveAvgPool2d((19,5))
input = torch.randn(1, 64, 8)
output = m(input)
output.shape

In [ ]:
class model(nn.Module):
    def __init__(self, input_size: int, n_channels: int, model_hyp: dict, classes: int):
        super(model, self).__init__()
        self.pe = PositionalEncoding(d_model=n_channels, max_len=input_size)
        self.ae = resnet18(n_channels=n_channels, groups=n_channels, num_classes=classes, d_model=model_hyp['d_model'])
        self.transformer_encoder = transformer_classifier(input_size, n_channels, model_hyp, classes)

    def forward(self, x):
        z = self.pe(x)
        z = z.transpose(-1,-2)
        z = self.ae(z)
        z = self.transformer_encoder(z)
        return z

In [ ]:
if __name__ == "__main__":
    logger = logging.getLogger(__name__)  # Use the current module's name
#     logging.basicConfig(filename=f'./logs/{datetime.now().strftime("%y%m%d%H%M")}_resnet18.log',level=logging.DEBUG)
    logging.basicConfig(level=logging.DEBUG)
    # logger.setLevel(logging.DEBUG)
    handler = logging.StreamHandler()
    # formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    # handler.setFormatter(formatter)
    logger.addHandler(handler)

    classifier = model(input_size=Configs['input_size'],
                                        n_channels = Configs['n_channels'],
                                        model_hyp=Configs['model'],
                                        classes=len(Configs['dataset']['classes'])).to('cuda')

In [ ]:
for m in classifier.modules():
    print(m)

In [ ]:
for m in classifier.parameters():
    print(m.dim(),m.shape, type(m))